### OCI Data Science - Explore data, Join, and clean

Before using this notebook users have to download neccessary data. We've collected/downloaded the data in advance by utilizing `00 pull data.ipunb` notebook and saved them to `data` folder.

This helps to save time, although data-download may take a few hours.

## Contents:

* <a href='#intro'>Introduction</a>
* <a href='#weatherdata'>Weather Data</a>
* <a href='#lapsdata'>Laps Data</a>
* <a href='#resultdata'>Results Data</a>
* <a href='#Joindata'>Merging Weather and Lap Data</a>
* <a href='#plotsamplelapdata'>Plotting Sample Lap Data</a>

<a id='intro'></a>
## Introduction

Data has already been saved in **data** folder. The data that interests us has `.pickle` extension, and are currently in raw format, which means that it requires pre-processing and preparation before performing feature extraction and model building.

This notebook only focuses on what's in the raw data.

#### Load packages

In [1]:
path = '/home/datascience/RedBull-Racining-TimeToPit/notebooks'
data_path = '../../RedBull-Racining-TimeToPit/data/'

In [2]:
# Imports
import os
os.chdir(path)
import pandas as pd
import logging
import json
import pickle
import requests
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [12]:
pd.__version__

'1.3.5'

In [11]:
def concat_data_and_save(data_source, data_path):
    '''
    e.g. data_source is weather
    1. finds all pkl files in the data_path folder
    2. concat all
    3. saves the in 'data_source'.pickle
    '''
    data = []
    files = os.listdir(data_path)
    file = [x for x in files if x.endswith('.pkl') if data_source in x]
    
    for fl in file:
        print(fl)
        file = open('{}{}'.format(data_path, fl), 'rb')
        data.extend(pickle.load(file))
    data = pd.concat(data, axis=0)

    file = open('{}{}_data.pickle'.format(data_path, data_source), 'wb')
    pickle.dump(data,file)
    return

In [7]:
concat_data_and_save('weathers', data_path)
concat_data_and_save('laps', data_path)
concat_data_and_save('results', data_path)

weathers_2022.pkl


AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/opt/conda/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>

<a id='weatherdata'></a>
## Weather Data
Weather data is available at the session level, and its time stamp is not aligned with laps-data.


In [8]:
file = open('{}{}'.format(data_path, 'weathers_data.pickle'),'rb')
df_weather = pickle.load(file)
# add eventYear to the dataset
df_weather['eventYear'] = pd.DatetimeIndex(df_weather['EventDate']).year # get the year

df_weather.head(2)

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/opt/conda/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>

In [ ]:
# average_weather: Get only race sessions, group them by event name, session and date, and get the mean for each one of the groups created for the rest of variables.
# avg. airtemp, humidity, pressure...
ave_weather = df_weather[df_weather['session']=='Race'].groupby(['EventName','session','EventDate']).mean().reset_index().sort_values(['EventName'])
ave_weather.head(5)

In [ ]:
# exercise: display average weather conditions for the coldest raining races in 2021
exercise_weather = df_weather[
    (df_weather['session']=='Race') &
    (df_weather['Rainfall']==True) &
    (df_weather['eventYear']==2021)
].groupby(['EventName', 'eventYear', 'RoundNumber']).mean().sort_values('AirTemp', ascending=True)
exercise_weather.head(5)

### Example: Track temp, ambient temp and humidity

In [ ]:
# Visualize track temperature, air temperature and humidity %.
# Valuable because tire life/performance depends on the track temperature
def plot_weather(d_f):

    fig = plt.figure(figsize=(12,2))
    ax = fig.add_axes([0,0,1,1])
    labels = [d_f.loc[x,['EventName']].values[0] +'_'+ 
              str(pd.DatetimeIndex(d_f.loc[x,['EventDate']]).year.values[0]) 
              for x in range(len(d_f))]
    x = np.arange(len(labels) )
    ax.bar(x + 0.00, d_f['AirTemp'], color='b', width=0.10)
    ax.bar(x + 0.25, d_f['TrackTemp'], color='r', width=0.10)
    ax.bar(x + 0.50, d_f['Humidity'], color='g', width=0.10)
    ax.set_xticks(x, labels, rotation=45, fontsize =10)
    ax.legend(['AirTemp','TrackTemp','Humidity'])
    ax.grid()
    plt.title('average weather parameters at different sessions')
    plt.show()
    return


plot_weather(ave_weather[(ave_weather['EventName']=='Austrian Grand Prix') | 
    (ave_weather['EventName']=='Abu Dhabi Grand Prix')].reset_index())

<a id='resultdata'></a>
## Results Data

Load data and select only `Red Bull Racing` data.

Drop non-race and non-qual rows from results data and keeping the following columns:
`['Position','FullName','Q1Sec','Q2Sec','Q3Sec', 'RoundNumber','EventName','eventYear','session','TimeSec']`

In [ ]:
file = open('{}{}'.format(data_path, 'results_data.pickle'),'rb')
df_res = pickle.load(file)

df_res['eventYear'] = pd.DatetimeIndex(df_res['EventDate']).year
df_res['Q1Sec'] = df_res['Q1']/np.timedelta64(1, 's')
df_res['Q2Sec'] = df_res['Q2']/np.timedelta64(1, 's')
df_res['Q3Sec'] = df_res['Q3']/np.timedelta64(1, 's')
df_res['TimeSec'] = df_res['Time']/np.timedelta64(1, 's')

## Selct only Red Bull Racing
df_res = df_res[df_res['TeamName']=='Red Bull Racing'] 

## keep only race and qualifier data
df_res = df_res[(df_res['session']=='Race') | (df_res['session']=='Qualifying')]
print('df_res size is: ', df_res.shape)
print('-'*100,'\n')

df_res= df_res[['Position','GridPosition','FullName','Q1Sec','Q2Sec','Q3Sec','RoundNumber','Abbreviation',
                'EventName','eventYear','session','Time','TimeSec','TeamName', 'EventDate']]

# Display round number 5, sort by position and year
df_res[df_res['RoundNumber']==5].sort_values(['Position','eventYear','Abbreviation'], ascending=False)

<a id='lapsdata'></a>
## Laps Data

Load data and select only `Red Bull Racing`. 

In [ ]:
file = open('{}{}'.format(data_path, 'laps_data.pickle'),'rb')

# Columns to drop
drop_cols = ['Sector1Time','Sector2Time','Sector3Time',
             'Sector1SessionTime','Sector2SessionTime','Sector3SessionTime',
            'SpeedI1','SpeedI2','SpeedFL', 'IsAccurate']

df_laps = pickle.load(file).drop(drop_cols,axis=1).reset_index(drop=True)

df_laps['eventYear'] = pd.DatetimeIndex(df_laps['EventDate']).year
df_laps['lapTimeSec'] = df_laps['LapTime']/np.timedelta64(1, 's')

# select only RedBull laps
df_laps = df_laps[df_laps['Team']=='Red Bull Racing']
df_laps.head()

#### Check Best Qualifying Times

Check if best Qual time from `results-data` is the same as best Qual time from `lap-data`
Best lap-time from lap Data. are we expecting to see the same best time from `results-data`?

In [ ]:
# get the minimum lap time for each event, year, session, driver -> minTime
best_Q = df_laps.groupby(['EventName','eventYear','session','Driver'])['lapTimeSec'].min().reset_index(name='minTime')

# example visualization: filter only from (race or qualifying), Abu Dhabi GP and check Sergio's minTimes.
best_Q[((best_Q['session']=='Race')|(best_Q['session']=='Qualifying') ) & 
(best_Q['EventName']=='Abu Dhabi Grand Prix') &
(best_Q['Driver']=='PER')].sort_values(['session'])
# note only 1 event appearing: it's because 2021 was the first year where this GP happened.

In [ ]:
# example visualization: filter only from (race or qualifying), Abu Dhabi GP, check Sergio's times for each session.
df_res[['TeamName', 'session','EventDate','eventYear','Q1Sec','Q2Sec','Q3Sec','TimeSec']][((df_res['session']=='Race')|(df_res['session']=='Qualifying') ) & 
(df_res['EventName']=='Abu Dhabi Grand Prix') &
(df_res['Abbreviation']=='PER')].sort_values(['session'])

# check why we have NaN values: Sergio had a DNF in the race.

Best Qual time from results-data and lap-data are the same! 
We will be using best Qual time as a feature from lap-data. 

(82.947s for Q3, vs. 82.947 minTime above.)

### End of data exploration